In [ ]:
# Useful to have
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Need for messing with the Fashion MNIST dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D

# Fitting Models to MNIST Fashion Data Set
This exercise will show you how to use TF to build a deep NN model that classifies elements of the Fashion MNIST data set.

In [ ]:
fashion_mnist_ds = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist_ds.load_data()
# From tutorial code - here are the names of the labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(X_train[0,:,:], cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
print("{0}...or, '{1}' ".format(y_train[0], class_names[y_train[0]]))

**Note**: NNs need to have data scaled between $[0,1]$ or $[-1,1]$.

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Look at a few of them...
plt.figure(figsize=(8,10))
num = 5
idxs = np.random.choice(X_train.shape[0], num)
i = 1
for idx in idxs:
    plt.subplot(1,num,i)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[idx], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[idx]])
    i += 1
plt.show()

In [ ]:
# And finally, let's convert the labels into one-hot encoding!
y_train_onehot = keras.utils.to_categorical(y_train, num_classes=len(class_names))
y_test_onehot = keras.utils.to_categorical(y_test, num_classes=len(class_names))
rand_idx = np.random.randint(y_train.shape[0])
# Example:
print("Original integer: {0}\nAs one-hot vector: {1}".format(y_train[rand_idx], y_train_onehot[rand_idx]))

Let's build up a model with the Tensorflow's `keras.Sequential` python API.
There are several [layer types](https://www.tensorflow.org/api_docs/python/tf/keras/layers) we can use.
To start, let's build a standard multi-layer perceptron, since we know theoretically all we need is a single hidden layer to approximate any continuous function.

**Note: MLPs do not operate on 2d data!**
We need to add a pass through later called `Flatten` to convert the 28x28 images into a 784 dimensional vector.

In [ ]:
img_shape = X_train[0].shape
num_elements = img_shape[0]*img_shape[1]
print("An image is a vector with {0} x {1} = {2} elements".format(img_shape[0], img_shape[1], num_elements))
fashion_classifier_model = keras.Sequential()
# "Dense" is the same as "Fully Connected"
fashion_classifier_model.add(Flatten(input_shape=img_shape))
fashion_classifier_model.add(Dense(int(num_elements / 2), activation='relu'))
fashion_classifier_model.add(Dense(10, activation='softmax')) # These are 10 logits.

Let's see what our model really outputs with some image...

In [ ]:
print(fashion_classifier_model.predict(X_train[0:1]))
print(y_train_onehot[0:1])

The model is built, now you need to pick a [loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses). 
The categories are represented with integers, so we will need to use Categorical Cross Entropy.

In [ ]:
loss_func = keras.losses.CategoricalCrossentropy()

In [ ]:
# Build the end-to-end optimization structure
fashion_classifier_model.compile(optimizer='adam',
                                loss=loss_func,
                                metrics=['accuracy'])

In [ ]:
# We invoke the fitting process: Graident Descent!
fashion_classifier_model.fit(X_train, y_train_onehot, epochs=1, batch_size=32)

In [ ]:
# Now you need to evaluate on the test set to see how you did!
fashion_classifier_model.evaluate(X_test, y_test_onehot, verbose=2)

Not bad! But let's look at the model structure. How many weights do we have?

In [ ]:
fashion_classifier_model.summary()

In [ ]:
# Let's fire up a prediction
test_img = X_test[0:1]
print("Array of scores - we can consider each value a confidence.")
pred = fashion_classifier_model.predict(test_img)
print(pred)

In [ ]:
# To extract the most likely class, we take the argmax!
pred_idx = np.argmax(fashion_classifier_model.predict(test_img))
print("We predicted input image as {0} with confidence {1}".format(class_names[pred_idx], pred[0][pred_idx]))

## Convolutional NN Model
In this example, we swap out MLP for CNNs. They are generally more efficient on 2D data. 

In [ ]:
# Have to reshape the data, since convolutional layers are expecting images with channels.
X_train_cnn = X_train.reshape(60000, 28, 28, 1)

In [ ]:
fashion_classifier_cnnmodel = keras.Sequential()
# Convolutional networks couple Convolution layers with pooling to boost efficiency.
fashion_classifier_cnnmodel.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)))
fashion_classifier_cnnmodel.add(MaxPool2D(pool_size=2))
fashion_classifier_cnnmodel.add(Flatten())
fashion_classifier_cnnmodel.add(Dense(10, activation='softmax'))

In [ ]:
fashion_classifier_cnnmodel.summary()

In [ ]:
# SAME training process...just a different model.
fashion_classifier_cnnmodel.compile(optimizer='adam',
                                    loss=loss_func,
                                    metrics=['accuracy'])

In [ ]:
fashion_classifier_cnnmodel.fit(X_train_cnn, y_train_onehot, epochs=1, batch_size=32)

In [ ]:
X_test_cnn = X_test.reshape(10000, 28, 28, 1)
fashion_classifier_model.evaluate(X_test_cnn, y_test_onehot, verbose=2)